# File Setup

In [1]:
#dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import sys
import gmaps
from pprint import pprint

#plot style
plt.style.use('dark_background')

#api-key in different folder, so need to adjust filepath
sys.path.append("/Users/Chrismus Time/Desktop/upenn-datacamp-2021/homework-repo/python-api-challenge")

#import keys
from config import gkey
from config import api_key

#configure key
gmaps.configure(api_key = gkey)

#github commands
# jupyter nbextension enable --py gmaps
# jupyter nbextension enable --py widgetsnbextension

# Dataframe Setup

In [2]:
#import data from created city csv
city_df = pd.read_csv("../WeatherPy/dataframe.csv")

#clean up data
clean_df = city_df.dropna(how = 'any').reset_index().drop(['index', 'Unnamed: 0'], axis = 1)
clean_df.head()

,City,Country,Latitude,Longitude,Temp (F),Humidity (%),Clouds (%),Wind Speed (mph)
0,tasiilaq,gl,65.613456,-37.633570,28.56,76.0,69.0,5.23
1,busselton,au,-33.651646,115.347334,46.04,81.0,1.0,8.12
2,albany,au,42.652579,-73.756232,48.85,74.0,1.0,1.99
3,cherskiy,ru,68.742677,161.350784,-16.02,94.0,91.0,0.69
4,west wendover,us,40.739097,-114.073345,43.81,81.0,1.0,0.00


# Plotting Humidity Heatmap

In [3]:
#figure settings
fig_settings = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#create fig
fig = gmaps.figure(layout = fig_settings)

#generating heatmap variables
coordinates = list(zip(clean_df['Latitude'].tolist(), clean_df['Longitude'].tolist()))
humidity_values = clean_df['Humidity (%)']
max_intensity = clean_df['Humidity (%)'].max()
point_radius = 7

#creating marker layer
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity_values, dissipating = True, max_intensity = max_intensity, point_radius = point_radius)

#testing the humidity to see why so much red
print(f"Humidity: mean {round(clean_df['Humidity (%)'].mean(),2)}, median {clean_df['Humidity (%)'].median()}, mode {clean_df['Humidity (%)'].mode()}.")

#adding markers to map
fig.add_layer(heat_layer)
fig

Humidity: mean 70.54, median 75.0, mode 0    91.0
dtype: float64.


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

# Plotting Ideal Heatmap

### Narrowing down ideal conditions

In [4]:
#using custom qualifications for personal ideal
ideal_df = clean_df.loc[(clean_df['Temp (F)'] < 77) &
                        (clean_df['Temp (F)'] > 60) &
                        (clean_df['Humidity (%)'] < 50) &
                        (clean_df['Wind Speed (mph)'] < 10) &
                        (clean_df['Clouds (%)'] > 10)]
print(f"There are only {len(ideal_df['City'])} ideal cities out of {len(clean_df['City'])}.")
ideal_df = ideal_df.reset_index().drop('index', axis = 1)
ideal_df.head()

There are only 4 ideal cities out of 452.


,City,Country,Latitude,Longitude,Temp (F),Humidity (%),Clouds (%),Wind Speed (mph)
0,lubango,ao,-14.918614,13.532123,76.55,34.0,100.0,1.23
1,manbij,sy,36.535285,37.967865,61.57,44.0,38.0,6.89
2,rawson,ar,-43.299858,-65.099487,74.08,26.0,98.0,8.90
3,waghai,in,20.773729,73.497566,71.22,46.0,43.0,5.68


### Fetching data for closest hotels

In [5]:
#setting variables for ideal heatmap
gplaces_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
gplaces_parameters = {
    'keyword' : 'hotel',
    'location' : '',
    'radius' : 5000,
    'key' : gkey
}

#setting lists/variables for forloop
found = False
hotel_names, hotel_lat, hotel_long = [], [], []
hotel_meta_list = [hotel_names, hotel_lat, hotel_long]
functions_list = ["hotel_response['results'][0]['name']", "hotel_response['results'][0]['geometry']['location']['lat']", "hotel_response['results'][0]['geometry']['location']['lng']"]

In [6]:
#requests hotel of ideal locations
for i in range(len(ideal_df['City'])):
    gplaces_parameters['location'] = f"{ideal_df['Latitude'].tolist()[i]},{ideal_df['Longitude'].tolist()[i]}"
    hotel_response = requests.get(gplaces_url, params = gplaces_parameters).json()
    for j in range(len(hotel_meta_list)):
        try:
            hotel_meta_list[j].append(eval(functions_list[j]))
            found = True
        except:
            hotel_meta_list[j].append(np.NaN)
    if (found == True):
        print(f"{ideal_df['City'].tolist()[i]} has at least one matching result.")
    else:
        print(f"{ideal_df['City'].tolist()[i]} has no matching results.")
    found = False #resetting for the next city

lubango has at least one matching result.
manbij has at least one matching result.
rawson has at least one matching result.
waghai has no matching results.


### Creating the new dataframe with hotel data

In [7]:
#variables for populating
new_columns = ['Closest Hotel', 'Hotel Latitude', 'Hotel Longitude']

#populating dataframe
for i in range(len(new_columns)):
    ideal_df[new_columns[i]] = hotel_meta_list[i]

#displaying new dataframe
ideal_df.head()

,City,Country,Latitude,Longitude,Temp (F),Humidity (%),Clouds (%),Wind Speed (mph),Closest Hotel,Hotel Latitude,Hotel Longitude
0,lubango,ao,-14.918614,13.532123,76.55,34.0,100.0,1.23,Chik-Chik Lubango,-14.916518,13.499923
1,manbij,sy,36.535285,37.967865,61.57,44.0,38.0,6.89,Manbej City Hotel,36.527127,37.951116
2,rawson,ar,-43.299858,-65.099487,74.08,26.0,98.0,8.90,Hosteria Sampedro,-43.301509,-65.104922
3,waghai,in,20.773729,73.497566,71.22,46.0,43.0,5.68,NaN,NaN,NaN


In [8]:
#create info box format
info_box_format = """
<dl>
<dt><b>Name</b></dt><dd>{Closest Hotel}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
</dl>
"""

#populating format for each city
hotel_info = [info_box_format.format(**row) for index, row in ideal_df.iterrows()]
#QUESTION FOR SELF: Review^, BCS helped me with this formula

In [9]:
#generating markers variables
coordinates = list(zip(ideal_df['Latitude'].tolist(), ideal_df['Longitude'].tolist()))

#creating marker layer
marker_layer = gmaps.marker_layer(coordinates, info_box_content = hotel_info)

#adding markers to map
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…